In [49]:
from pydantic import BaseModel
from typing import List, Optional
import os
import json

class Player(BaseModel):
    name: str
    buchholz_score: int
    points_for: int
    points_against: int
    wins: int
    losses: int
    had_bye: bool = False
    

class Game(BaseModel):
    player1: str
    player2: Optional[str]
    points1: int = 0
    points2: int = 0

class Round(BaseModel):
    number: int
    games: List[Game]
    preset: bool = False

class Tournament(BaseModel):
    name: str
    folder: str
    rounds: List[Round]
    players: List[Player]

    def __init__(self, name: str, players: List[str]):
        name = name
        folder = os.path.join("data", name)
        os.makedirs(folder, exist_ok=True)

        super().__init__(rounds=[], players=[], folder=folder, name=name)

        for p in players:
            self.players.append(Player(
                name=p,
                buchholz_score=0,
                points_for=0,
                points_against=0,
                wins=0,
                losses=0,
            ))

        self.save_state()


    def save_state(self):
        data = self.model_dump()
        data["rounds"] = []  # prevent duplication

        with open(os.path.join(self.folder, "tournament.json"), "w") as f:
            json.dump(data, f, indent=2)

    def all_games(self):
        return [g for r in self.rounds for g in r.games]



    @classmethod
    def load(cls, name: str):
        with open(os.path.join("data", name, "tournament.json")) as f:
            return cls.model_validate_json(f.read())
        
    def load_rounds(self):
        self.rounds = []
        i = 1
        while True:
            path = self.round_path(i)
            if not os.path.exists(path):
                break
            with open(path) as f:
                r = Round.model_validate_json(f.read())
                self.rounds.append(r)
            i += 1
    
    def load_round(self, number: int) -> Optional[Round]:
        path = self.round_path(number)

        if not os.path.exists(path):
            return None

        with open(path) as f:
            return Round.model_validate_json(f.read())

    def all_rounds(self) -> List[Round]:
        rounds = []
        i = 1

        while True:
            r = self.load_round(i)
            if not r:
                break
            rounds.append(r)
            i += 1

        return rounds

    def save_round(self, round: Round):
        # Replace if round with same number exists
        self.rounds = [r for r in self.rounds if r.number != round.number]
        self.rounds.append(round)

        with open(self.round_path(round.number), "w") as f:
            f.write(round.model_dump_json(indent=2))

    def round_path(self, number: int):
        return os.path.join(self.folder, f"round_{number:03}.json")
    
    def reset_player_points_and_records(self) -> int:
        for player in self.players:
            player.points_for = 0
            player.points_against = 0
            player.wins = 0
            player.losses = 0
            player.buchholz_score = 0

    def calculate_points_and_win_loss(self):
        # reset first
        self.reset_player_points_and_records()

        for game in self.all_games():
            p1 = next(p for p in self.players if p.name == game.player1)
            if game.player2:
                p2 = next(p for p in self.players if p.name == game.player2)
            else:
                # BYE counts as a win
                p1.wins += 1
                continue

            if game.points1 > game.points2:
                p1.wins += 1
                p2.losses += 1
            elif game.points1 < game.points2:
                p2.wins += 1
                p1.losses += 1

            p1.points_for += game.points1
            p1.points_against += game.points2
            p2.points_for += game.points2
            p2.points_against += game.points1

        self.calculate_buchholz_scores()



    def calculate_buchholz_scores(self) -> int:
        for player in self.players:
            score = 0
            for game in self.all_games():
                if game.player1 == player.name:
                    opponent = game.player2
                elif game.player2 == player.name:
                    opponent = game.player1
                else:
                    continue

                opponent = self.find_player_by_name(opponent)
                if opponent is None:
                    continue
                score += opponent.wins
            player.buchholz_score = score

    def have_played(self, p1: Player, p2: Player) -> bool:
        for game in self.all_games():
            if (
                (game.player1 == p1.name and game.player2 == p2.name)
                or
                (game.player1 == p2.name and game.player2 == p1.name)
            ):
                return True
        return False
    

    def find_player_by_name(self, name: str) -> Optional[Player]:
        for player in self.players:
            if player.name == name:
                return player
        return None


    def create_and_show_next_round_pairings(self) -> List[Game]:

        self.reset_player_points_and_records()
        self.calculate_points_and_win_loss()
        self.calculate_buchholz_scores()

        self.players.sort(
            key=lambda p: (p.wins, p.buchholz_score),
            reverse=True,
        )

        rounds = self.all_rounds()
        round_number = max((r.number for r in rounds), default=0) + 1

        unpaired = self.players[:]
        games = []

        # ---------- BYE ----------
        if len(unpaired) % 2 == 1:
            for player in reversed(unpaired):
                if not player.had_bye:
                    bye_player = player
                    break
            else:
                bye_player = unpaired[-1]

            unpaired.remove(bye_player)
            bye_player.had_bye = True

            games.append(Game(
                player1=bye_player.name,
                player2=None,
                points1=1,
                points2=0
            ))

        # ---------- pairing ----------
        while len(unpaired) > 1:
            p1 = unpaired.pop(0)

            opponent_index = None
            for i, p2 in enumerate(unpaired):
                if not self.have_played(p1, p2):
                    opponent_index = i
                    break

            if opponent_index is None:
                opponent_index = 0

            p2 = unpaired.pop(opponent_index)

            games.append(Game(
                player1=p1.name,
                player2=p2.name
            ))

        round_obj = Round(number=round_number, games=games)
        self.save_round(round_obj)

        return games
    

    def get_standings(self) -> List[Player]:
        self.load_rounds()
        self.reset_player_points_and_records()
        self.calculate_points_and_win_loss()
        self.calculate_buchholz_scores()

        return sorted(
            self.players,
            key=lambda p: (p.wins, p.buchholz_score),
            reverse=True,
        )


In [55]:
tourn = Tournament("test_tournament", ["Alice", "Bob", "Charlie", "David", "Eve"])

In [61]:
tourn.get_standings()

[Player(name='Eve', buchholz_score=1, points_for=4, points_against=2, wins=2, losses=0, had_bye=True),
 Player(name='David', buchholz_score=1, points_for=8, points_against=4, wins=2, losses=0, had_bye=True),
 Player(name='Alice', buchholz_score=2, points_for=6, points_against=5, wins=1, losses=1, had_bye=False),
 Player(name='Charlie', buchholz_score=2, points_for=2, points_against=4, wins=1, losses=1, had_bye=True),
 Player(name='Bob', buchholz_score=3, points_for=3, points_against=8, wins=0, losses=2, had_bye=False)]

In [62]:
tourn.create_and_show_next_round_pairings()

[Game(player1='Bob', player2=None, points1=1, points2=0),
 Game(player1='Eve', player2='David', points1=0, points2=0),
 Game(player1='Alice', player2='Charlie', points1=0, points2=0)]